# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [55]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
from scipy.stats import ttest_ind
%matplotlib inline

In [2]:
#Your code here
df =pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [ ]:
df['id'].value_counts()

363314    2
368962    2
568156    2
267099    2
322257    2
         ..
419672    1
448358    1
761703    1
929641    1
196608    1
Name: id, Length: 6328, dtype: int64

In [30]:
# Get the value count for the action column
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

#### Define hypothesis
$H_o$ =There is no difference in the click-through rate between the experimental and control group

$H_a$ =There is difference in the click-through rate between the experimental and control group

In [80]:
# Calculate the click through rate
""" 
Filter the data for the two groups
CTR = Total clicks/Total views
"""
Expr =df.loc[(df['group']=="experiment")]
click_expr =len(Expr.loc[Expr['action']=="click"])
view_expr =len(Expr.loc[Expr['action']=="view"])

Contr =df.loc[(df['group']=="control")]
click_contr =len(Contr.loc[Contr['action']=="click"])
view_contr =len(Contr.loc[Contr['action']=="view"])

CTR_expr = click_expr/view_expr

CTR_contr = click_contr/view_contr

CTR_expr, CTR_contr

# Calculate the pooled proportion
p = (click_contr+click_expr)/(view_contr+view_expr)
p

# Calculate the z-score
z =(CTR_expr-CTR_contr)/np.sqrt(p*(1-p)*(1/view_contr+1/view_expr))
z
# Get the p-value 
p_value = 2*(1-st.norm.cdf(z))
p_value

0.008830075576595764

From the statistical test we get a p-value that is less than $\alpha = 0.05$ we reject the null hypothesis and conclude that the click through rate of the experimental is different from the control group. 

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [96]:
#Your code here
n1=len(df.loc[(df['group']=="experiment") & (df['action']=="view")])
n2=len(df.loc[(df['group']=="control") & (df['action']=="view")])
# Expected number (np)
Exp_number_exp =n1*CTR_contr
Exp_number_cont =n2*CTR_expr

Exp_number_exp,Exp_number_cont



(838.0168067226891, 1032.0747663551401)

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [102]:
#Your code here
std = np.sqrt(p*(1-CTR_contr)*n1)
std


25.185307971000725

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [103]:
#Your code here

# Compute the z-score
z_score=(click_expr-Exp_number_exp)/std
z_score



# get the Two-tailed p-value 
p_value =2*(1-st.norm.cdf(z_score))
p_value

0.00035312424489553074

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: Yeah the results actually match as we confirmed that their statistical difference between the click through rate of the experimental and control group

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.